## Project 4: Neural Network with TensorFlow

**Objective:**

Build a neural network to predict shipment times.

**Instructions**
1. Define a neural network architecture.
2. Train the neural network.
3. Evaluate the model's performance.

## Import the necessary labraries and frameworks.

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf
from tensorflow.keras.models import  Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import Input

import random

## Set seeds for reproducibility

In [5]:
# 1. Python built-in RNG
random.seed(30)

# 2. NumPy RNG
np.random.seed(30)

# 3. PyTorch CPU RNG
torch.manual_seed(30)

# 4. PyTorch GPU RNG (if using CUDA)
torch.cuda.manual_seed_all(30)

# 5. Force deterministic CUDA (optional, may slow you down)
torch.use_deterministic_algorithms(True)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Read the CSV file.

In [7]:
shipment_df = pd.read_csv('shipment_df.csv')

## Encoding Categorical Columns for Model Ingestion.

In [9]:
shipment_df = pd.get_dummies(shipment_df, columns=['transportation_modes', 'location', 'routes'], drop_first=True).astype('int64')

## Feature Engineering
#### Normalize numerical features

In [11]:
# 1) Identify your target(s) and features
shipmenty = 'shipping_times'
shipmentX = shipment_df.drop(columns=[shipmenty])

# 2) Select only the numeric columns in the features
num_cols = shipmentX.select_dtypes(include=['int64', 'float64']).columns
#    (this will pick up your distance, count, and one-hot dummy columns too)

# 3) Fit & transform only those numeric columns
scaler = StandardScaler()
shipmentX[num_cols] = scaler.fit_transform(shipmentX[num_cols])

shipmentX_scaled = shipmentX
shipmenty = shipment_df[shipmenty]

## Model training

# Train a deep learning model with tensorflow.

## Split the dataset

In [15]:
shipmentX_train, shipmentX_test, shipmenty_train, shipmenty_test = train_test_split(shipmentX_scaled, shipmenty, test_size=0.25, random_state=30)

## Define the neural network model

In [17]:

shipment_nn_model = Sequential([
    Input(shape=(shipmentX_train.shape[1],)), 
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)
])

# Compile and Train the model

In [19]:
# Compile the model
shipment_nn_model.compile(optimizer='adam', loss='mse')
# Train the model
shipment_nn_model.fit(shipmentX_train, shipmenty_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 3.2066 - val_loss: 0.3254
Epoch 2/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3162 - val_loss: 0.2921
Epoch 3/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2944 - val_loss: 0.2876
Epoch 4/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2871 - val_loss: 0.2817
Epoch 5/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2862 - val_loss: 0.2828
Epoch 6/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2803 - val_loss: 0.2837
Epoch 7/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2798 - val_loss: 0.2836
Epoch 8/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2822 - val_loss: 0.2779
Epoch 9/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2792 - val_loss: 0.2906
Epoch 10/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2810 - val_loss: 0.2861
Epoch 11/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2789 - val_loss: 0.2833
Epoch 12/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step

## Evaluate the performance of the models

In [21]:
shipment_loss = shipment_nn_model.evaluate(shipmentX_test, shipmenty_test)

print(f'Mean Squared Error: {shipment_loss}')

196/196 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2749
Mean Squared Error: 0.2766462564468384


## Conclusion

### Methods Used:

* Built 3-layer Sequential model (64-32-1 neurons)
* Used Adam optimizer and MSE loss
* Trained for 50 epochs (batch_size=32)


### Results: Achieved MSE: 0.276

* The result is Comparable to Random Forest regression and gradient boosters MSE in Project 3
* Slight improvement over the gradient boosting and Random Forest models
* Performed slightly lower than the gradient boosters
* Potential underfitting observed

### Improvement Suggestions:

* I may need to add dropout layers for regularization
* Implement early stopping to prevent overfitting
* And experiment with different activation functions